HUMAN VS GENERATED FROM AI USING TFIDF,BAGGING,BOOSTING,STACKING,BERT,RNN(LSTM)
TOOK REVIEW FROM IMDB AND FOR HUMAN
AND ASK GEMINI TO GENERATE REVIEW NORMAL AND HUMANIZED REVIEW
TOTAL SAMPLE:- 150=75+75

In [ ]:
#tfidf vector and logistic regression traditional ml

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# 1. Load the Dataset
df = pd.read_csv('imdb_human_vs_gen_review.csv', encoding='latin-1')

print(df.head())
print("\nValue Counts:")
print(df['Label'].value_counts())

# 2. Split Data into Training and Testing Sets
X = df['Text']
y = df['Label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 3. Create and Train the TF-IDF Vectorizer
# This converts text into a numerical format
vectorizer = TfidfVectorizer(stop_words='english', max_features=50000)

# Fit the vectorizer on the training data and transform it
X_train_tfidf = vectorizer.fit_transform(X_train)

# Only transform the test data (using the vocabulary from the training data)
X_test_tfidf = vectorizer.transform(X_test)

# 4. Train the Logistic Regression Model
model = LogisticRegression(solver='liblinear', random_state=42)
model.fit(X_train_tfidf, y_train)

# 5. Evaluate the Model
# Make predictions on the test set
y_pred = model.predict(X_test_tfidf)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"\nModel Accuracy: {accuracy:.4f}")

# Print the classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=['Human', 'AI']))

# Print the confusion matrix
print("\nConfusion Matrix:")
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Human', 'AI'], yticklabels=['Human', 'AI'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()


# 6. Function to Predict on New Text
def predict_text(text):
    # Transform the new text using the fitted vectorizer
    text_tfidf = vectorizer.transform([text])
    # Predict the label
    prediction = model.predict(text_tfidf)
    prediction_proba = model.predict_proba(text_tfidf)

    if prediction[0] == 0:
        return f"Prediction: Human (Probability: {prediction_proba[0][0]:.2f})"
    else:
        return f"Prediction: AI (Probability: {prediction_proba[0][1]:.2f})"

# --- Example Predictions ---
print("\n--- Making Predictions on New Text ---")
human_text_example = "Aashiqui 2,a truly amazing love story. It was a pleasure watching such a beautifully directed movie with a great and different story. Fantastic work by the not so experienced new faces Shraddha kapoor and Aditya Roy kapoor which made a very beautiful couple on stage. It wasn't the usual same old love story script but very well written and executed script. The emotions and truly incredible songs put up at appropriate moments were the charm! Shraddha kapoor's roles in her previous films like luv ka the end, teen Patti were not as effective and couldn't bring her into the limelight. But this movie is surely going to be a great comeback for her in the Bollywood film industry.Same goes with Aditya Roy kapoor.A very effective n deeply heart touching love story which is a must watch if you like love story movies..!!"

ai_text_example = "Aashiqui 2 is a romantic musical drama that centers on the turbulent relationship between Rahul Jaykar, a famous but self-destructive singer, and Aarohi Shirke, a talented singer he discovers and mentors. The film's narrative is propelled by its critically acclaimed soundtrack, which effectively captures the central themes of love, fame, and sacrifice. The on-screen chemistry between Aditya Roy Kapur and Shraddha Kapoor is a significant strength, lending emotional weight to the story. However, the plot heavily borrows from the classic A Star Is Born archetype, and the protagonist's persistent battle with alcoholism can make for a frustrating viewing experience. Despite its narrative predictability, the film succeeds as a poignant and musically rich tragedy."

ai_humanized_example="Okay, let's be real about Aashiqui 2. I still remember walking out of the theater feeling emotionally drained, and honestly, it's all because of the music. The soundtrack for this movie is legendary, hands down. You can't hear the first few notes of Tum Hi Ho without getting all the feels, am I right? Aditya Roy Kapur and Shraddha Kapoor were just perfect as RJ and Aarohi. You totally buy into their love story, which makes his struggle with alcoholism so much more painful to watch. Yeah, the plot is basically a desi A Star Is Born, and there were times I wanted to shake RJ and tell him to get it together. But the emotions and the songs are so powerful that you kind of forgive its flaws. It’s a beautiful, heartbreaking movie that will definitely stick with you."


print(f"\nInput: '{human_text_example}'")
print(predict_text(human_text_example))

print(f"\nInput: '{ai_text_example}'")
print(predict_text(ai_text_example))

print(f"\nInput: '{ai_humanized_example}'")
print(predict_text(ai_humanized_example))

In [ ]:
#bagging

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report,accuracy_score

# Load your dataset
df = pd.read_csv('/content/imdb_human_vs_gen_review.csv', encoding='latin-1')
X = df['Text']
y = df['Label']

# Create TF-IDF features
vectorizer = TfidfVectorizer(max_features=50000, stop_words='english')
X_tfidf = vectorizer.fit_transform(X)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42, stratify=y)

# --- Train a Random Forest Model ---
# n_estimators is the number of trees (models) in the forest.
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf_model.fit(X_train, y_train)

# --- Evaluate the Model ---
y_pred = rf_model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"\nModel Accuracy: {accuracy:.4f}")

print("--- Random Forest (Bagging) Results ---")
print(classification_report(y_test, y_pred, target_names=['Human', 'AI']))

In [ ]:
# 6. Function to Predict on New Text
def predict_text(text):
    # Transform the new text using the fitted vectorizer
    text_tfidf = vectorizer.transform([text])
    # Predict the label
    prediction = rf_model.predict(text_tfidf)
    prediction_proba = rf_model.predict_proba(text_tfidf)

    if prediction[0] == 0:
        return f"Prediction: Human (Probability: {prediction_proba[0][0]:.2f})"
    else:
        return f"Prediction: AI (Probability: {prediction_proba[0][1]:.2f})"

# --- Example Predictions ---
print("\n--- Making Predictions on New Text ---")
human_text_example = "Aashiqui 2,a truly amazing love story. It was a pleasure watching such a beautifully directed movie with a great and different story. Fantastic work by the not so experienced new faces Shraddha kapoor and Aditya Roy kapoor which made a very beautiful couple on stage. It wasn't the usual same old love story script but very well written and executed script. The emotions and truly incredible songs put up at appropriate moments were the charm! Shraddha kapoor's roles in her previous films like luv ka the end, teen Patti were not as effective and couldn't bring her into the limelight. But this movie is surely going to be a great comeback for her in the Bollywood film industry.Same goes with Aditya Roy kapoor.A very effective n deeply heart touching love story which is a must watch if you like love story movies..!!"

ai_text_example = "Aashiqui 2 is a romantic musical drama that centers on the turbulent relationship between Rahul Jaykar, a famous but self-destructive singer, and Aarohi Shirke, a talented singer he discovers and mentors. The film's narrative is propelled by its critically acclaimed soundtrack, which effectively captures the central themes of love, fame, and sacrifice. The on-screen chemistry between Aditya Roy Kapur and Shraddha Kapoor is a significant strength, lending emotional weight to the story. However, the plot heavily borrows from the classic A Star Is Born archetype, and the protagonist's persistent battle with alcoholism can make for a frustrating viewing experience. Despite its narrative predictability, the film succeeds as a poignant and musically rich tragedy."

ai_humanized_example="Okay, let's be real about Aashiqui 2. I still remember walking out of the theater feeling emotionally drained, and honestly, it's all because of the music. The soundtrack for this movie is legendary, hands down. You can't hear the first few notes of Tum Hi Ho without getting all the feels, am I right? Aditya Roy Kapur and Shraddha Kapoor were just perfect as RJ and Aarohi. You totally buy into their love story, which makes his struggle with alcoholism so much more painful to watch. Yeah, the plot is basically a desi A Star Is Born, and there were times I wanted to shake RJ and tell him to get it together. But the emotions and the songs are so powerful that you kind of forgive its flaws. It’s a beautiful, heartbreaking movie that will definitely stick with you."


print(f"\nInput: '{human_text_example}'")
print(predict_text(human_text_example))

print(f"\nInput: '{ai_text_example}'")
print(predict_text(ai_text_example))

print(f"\nInput: '{ai_humanized_example}'")
print(predict_text(ai_humanized_example))

In [ ]:
#boosting

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier # You may need to run: !pip install xgboost
from sklearn.metrics import classification_report,accuracy_score

# Load your dataset
df = pd.read_csv('/content/imdb_human_vs_gen_review.csv', encoding='latin-1')
X = df['Text']
y = df['Label']

# Create TF-IDF features
vectorizer = TfidfVectorizer(max_features=50000, stop_words='english')
X_tfidf = vectorizer.fit_transform(X)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42, stratify=y)

# --- Train an XGBoost Model ---
xgb_model = XGBClassifier(n_estimators=100, random_state=42, use_label_encoder=False, eval_metric='logloss')
xgb_model.fit(X_train, y_train)

# --- Evaluate the Model ---
y_pred = xgb_model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"\nModel Accuracy: {accuracy:.4f}")

print("--- XGBoost (Boosting) Results ---")
print(classification_report(y_test, y_pred, target_names=['Human', 'AI']))

In [ ]:
# 6. Function to Predict on New Text
def predict_text(text):
    # Transform the new text using the fitted vectorizer
    text_tfidf = vectorizer.transform([text])
    # Predict the label
    prediction = xgb_model.predict(text_tfidf)
    prediction_proba = xgb_model.predict_proba(text_tfidf)

    if prediction[0] == 0:
        return f"Prediction: Human (Probability: {prediction_proba[0][0]:.2f})"
    else:
        return f"Prediction: AI (Probability: {prediction_proba[0][1]:.2f})"

# --- Example Predictions ---
print("\n--- Making Predictions on New Text ---")
human_text_example = "Aashiqui 2,a truly amazing love story. It was a pleasure watching such a beautifully directed movie with a great and different story. Fantastic work by the not so experienced new faces Shraddha kapoor and Aditya Roy kapoor which made a very beautiful couple on stage. It wasn't the usual same old love story script but very well written and executed script. The emotions and truly incredible songs put up at appropriate moments were the charm! Shraddha kapoor's roles in her previous films like luv ka the end, teen Patti were not as effective and couldn't bring her into the limelight. But this movie is surely going to be a great comeback for her in the Bollywood film industry.Same goes with Aditya Roy kapoor.A very effective n deeply heart touching love story which is a must watch if you like love story movies..!!"

ai_text_example = "Aashiqui 2 is a romantic musical drama that centers on the turbulent relationship between Rahul Jaykar, a famous but self-destructive singer, and Aarohi Shirke, a talented singer he discovers and mentors. The film's narrative is propelled by its critically acclaimed soundtrack, which effectively captures the central themes of love, fame, and sacrifice. The on-screen chemistry between Aditya Roy Kapur and Shraddha Kapoor is a significant strength, lending emotional weight to the story. However, the plot heavily borrows from the classic A Star Is Born archetype, and the protagonist's persistent battle with alcoholism can make for a frustrating viewing experience. Despite its narrative predictability, the film succeeds as a poignant and musically rich tragedy."

ai_humanized_example="Okay, let's be real about Aashiqui 2. I still remember walking out of the theater feeling emotionally drained, and honestly, it's all because of the music. The soundtrack for this movie is legendary, hands down. You can't hear the first few notes of Tum Hi Ho without getting all the feels, am I right? Aditya Roy Kapur and Shraddha Kapoor were just perfect as RJ and Aarohi. You totally buy into their love story, which makes his struggle with alcoholism so much more painful to watch. Yeah, the plot is basically a desi A Star Is Born, and there were times I wanted to shake RJ and tell him to get it together. But the emotions and the songs are so powerful that you kind of forgive its flaws. It’s a beautiful, heartbreaking movie that will definitely stick with you."


print(f"\nInput: '{human_text_example}'")
print(predict_text(human_text_example))

print(f"\nInput: '{ai_text_example}'")
print(predict_text(ai_text_example))

print(f"\nInput: '{ai_humanized_example}'")
print(predict_text(ai_humanized_example))

In [ ]:
#stacking

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Load your dataset
df = pd.read_csv('/content/imdb_human_vs_gen_review.csv', encoding='latin-1')
X = df['Text']
y = df['Label']

# Create TF-IDF features
vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
X_tfidf = vectorizer.fit_transform(X)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42, stratify=y)

# --- Define the Base Models ---
base_estimators = [
    ('nb', MultinomialNB()),
    ('rf', RandomForestClassifier(n_estimators=50, random_state=42))
]

# --- Define the Meta-Model (the 'manager') ---
# This model will make the final decision based on the base models' predictions.
meta_model = LogisticRegression()

# --- Create and Train the Stacking Classifier ---
stacking_model = StackingClassifier(estimators=base_estimators, final_estimator=meta_model, cv=5)
stacking_model.fit(X_train, y_train)

# --- Evaluate the Model ---
y_pred = stacking_model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"\nModel Accuracy: {accuracy:.4f}")

print("--- Stacking Results ---")
print(classification_report(y_test, y_pred, target_names=['Human', 'AI']))

In [ ]:
# 6. Function to Predict on New Text
def predict_text(text):
    # Transform the new text using the fitted vectorizer
    text_tfidf = vectorizer.transform([text])
    # Predict the label
    prediction = stacking_model.predict(text_tfidf)
    prediction_proba = stacking_model.predict_proba(text_tfidf)

    if prediction[0] == 0:
        return f"Prediction: Human (Probability: {prediction_proba[0][0]:.2f})"
    else:
        return f"Prediction: AI (Probability: {prediction_proba[0][1]:.2f})"

# --- Example Predictions ---
print("\n--- Making Predictions on New Text ---")
human_text_example = "Aashiqui 2,a truly amazing love story. It was a pleasure watching such a beautifully directed movie with a great and different story. Fantastic work by the not so experienced new faces Shraddha kapoor and Aditya Roy kapoor which made a very beautiful couple on stage. It wasn't the usual same old love story script but very well written and executed script. The emotions and truly incredible songs put up at appropriate moments were the charm! Shraddha kapoor's roles in her previous films like luv ka the end, teen Patti were not as effective and couldn't bring her into the limelight. But this movie is surely going to be a great comeback for her in the Bollywood film industry.Same goes with Aditya Roy kapoor.A very effective n deeply heart touching love story which is a must watch if you like love story movies..!!"

ai_text_example = "Aashiqui 2 is a romantic musical drama that centers on the turbulent relationship between Rahul Jaykar, a famous but self-destructive singer, and Aarohi Shirke, a talented singer he discovers and mentors. The film's narrative is propelled by its critically acclaimed soundtrack, which effectively captures the central themes of love, fame, and sacrifice. The on-screen chemistry between Aditya Roy Kapur and Shraddha Kapoor is a significant strength, lending emotional weight to the story. However, the plot heavily borrows from the classic A Star Is Born archetype, and the protagonist's persistent battle with alcoholism can make for a frustrating viewing experience. Despite its narrative predictability, the film succeeds as a poignant and musically rich tragedy."

ai_humanized_example="Okay, let's be real about Aashiqui 2. I still remember walking out of the theater feeling emotionally drained, and honestly, it's all because of the music. The soundtrack for this movie is legendary, hands down. You can't hear the first few notes of Tum Hi Ho without getting all the feels, am I right? Aditya Roy Kapur and Shraddha Kapoor were just perfect as RJ and Aarohi. You totally buy into their love story, which makes his struggle with alcoholism so much more painful to watch. Yeah, the plot is basically a desi A Star Is Born, and there were times I wanted to shake RJ and tell him to get it together. But the emotions and the songs are so powerful that you kind of forgive its flaws. It’s a beautiful, heartbreaking movie that will definitely stick with you."


print(f"\nInput: '{human_text_example}'")
print(predict_text(human_text_example))

print(f"\nInput: '{ai_text_example}'")
print(predict_text(ai_text_example))

print(f"\nInput: '{ai_humanized_example}'")
print(predict_text(ai_humanized_example))

In [ ]:
!pip install transformers[torch] datasets evaluate

In [ ]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from transformers import BertTokenizer, BertModel

#################################################################
### PHASE 1: TRAINING THE MODEL ON THE ENTIRE DATASET         ###
#################################################################

# --- 1. Load your Dataset ---
df = pd.read_csv('/content/imdb_human_vs_gen_review.csv', encoding='latin-1')

# --- 2. Initialize Pre-trained BERT Model and Tokenizer ---
print("Loading BERT model and tokenizer...")
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')# Loads the rules for converting text to numbers.
bert_model = BertModel.from_pretrained('bert-base-uncased')#Loads the trained neural network that understands the meaning of those numbers.
print("BERT model loaded.")

# --- 3. Generate BERT Embeddings (Efficiently in Batches) ---
def get_bert_embeddings_batch(texts, model, tokenizer, batch_size=16):
    """Processes texts in batches for much faster embedding generation."""
    model.eval()  #It switches the BERT model into "evaluation mode."
    device = "cuda" if torch.cuda.is_available() else "cpu"   #The first line checks if your computer has a powerful GPU available (NVIDIA GPUs use a technology called CUDA)
    model.to(device)   #moves the entire, massive BERT model onto that chosen device.
    all_cls_embeddings = []    #creates an empty list. This list will act as a container. As the function processes your reviews in batches, it will generate the numerical embeddings for each review and store them in this list.

    print(f"\nGenerating BERT embeddings in batches of {batch_size} on device: {device}")
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size]    #This line simply grabs the current chunk (or batch) of 16 reviews from your main list.
        inputs = tokenizer(batch_texts, return_tensors='pt', truncation=True, padding=True, max_length=128).to(device)
        with torch.no_grad():
            outputs = model(**inputs)
        cls_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()
        all_cls_embeddings.extend(cls_embeddings)
        print(f"  Processed {min(i + batch_size, len(texts))}/{len(texts)} reviews")

    return np.array(all_cls_embeddings)

# Get the list of texts from your 'Text' column
reviews_list = df['Text'].tolist()
X_bert = get_bert_embeddings_batch(reviews_list, bert_model, tokenizer)

# Get the labels from your 'Label' column
y = df['Label']
print("Embeddings generated.")


# --- 4. Split Data ---
X_train, X_test, y_train, y_test = train_test_split(X_bert, y, test_size=0.3, random_state=42, stratify=y)


# --- 5. Train the Logistic Regression Model ---
print("\n--- Training the Classification Model ---")
model_lr_on_bert = LogisticRegression(max_iter=1000)
model_lr_on_bert.fit(X_train, y_train)
print("--- Training Complete ---")


# --- 6. Display Training Results ---
y_pred = model_lr_on_bert.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("\n--- Model Evaluation Results ---")
print(f"Model Accuracy: {accuracy:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=['Human', 'AI'], zero_division=0))



In [ ]:

#################################################################
### PHASE 2: PREDICTING ON NEW, UNSEEN REVIEWS                ###
#################################################################

# --- 7. Define a Reusable Prediction Function ---
def predict_new_review(text, bert_model, tokenizer, logistic_regression_model):
    """
    Takes a raw text string and classifies it using the trained models.
    """
    device = "cuda" if torch.cuda.is_available() else "cpu"
    bert_model.to(device)
    bert_model.eval()

    # Get the BERT embedding for the single new text
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=128).to(device)  #The tokenizer converts your raw text string into a sequence of numbers (Token IDs) that BERT can understand. It also makes sure the sequence is exactly 128 tokens long.
    with torch.no_grad():  #This is an optimization that tells the program we're only predicting, not training, which makes it faster.
        outputs = bert_model(**inputs)  #The numerical sequence is fed into the BERT model. BERT processes it and outputs a rich, contextual understanding of the text.
    embedding = outputs.last_hidden_state[:, 0, :].cpu().numpy()  #This line extracts the final numerical summary (the embedding) from BERT's complex output.

    # Use the trained Logistic Regression model to predict
    prediction = model_lr_on_bert.predict(embedding)
    prediction_proba = model_lr_on_bert.predict_proba(embedding)

    if prediction[0] == 0:
        confidence = prediction_proba[0][0]
        return f"Prediction: Human (Confidence: {confidence:.2f})"
    else:
        confidence = prediction_proba[0][1]
        return f"Prediction: AI (Confidence: {confidence:.2f})"

# --- 8. Use the Function to Classify New Reviews ---
print("\n\n--- Making Predictions on New Text ---")

# Example 1
review_1_human = "Aashiqui 2,a truly amazing love story. It was a pleasure watching such a beautifully directed movie with a great and different story. Fantastic work by the not so experienced new faces Shraddha kapoor and Aditya Roy kapoor which made a very beautiful couple on stage. It wasn't the usual same old love story script but very well written and executed script. The emotions and truly incredible songs put up at appropriate moments were the charm! Shraddha kapoor's roles in her previous films like luv ka the end, teen Patti were not as effective and couldn't bring her into the limelight. But this movie is surely going to be a great comeback for her in the Bollywood film industry.Same goes with Aditya Roy kapoor.A very effective n deeply heart touching love story which is a must watch if you like love story movies..!!"
result_1 = predict_new_review(review_1_human, bert_model, tokenizer, model_lr_on_bert)
print(f"\nReview to classify: '{review_1_human}'")
print(result_1)

# Example 2
review_2_ai = "Aashiqui 2 is a romantic musical drama that centers on the turbulent relationship between Rahul Jaykar, a famous but self-destructive singer, and Aarohi Shirke, a talented singer he discovers and mentors."
result_2 = predict_new_review(review_2_ai, bert_model, tokenizer, model_lr_on_bert)
print(f"\nReview to classify: '{review_2_ai}'")
print(result_2)

# Example 3
review_3_ai_human = "Okay, let's be real about Aashiqui 2. I still remember walking out of the theater feeling emotionally drained, and honestly, it's all because of the music. The soundtrack for this movie is legendary, hands down."
result_3 = predict_new_review(review_3_ai_human, bert_model, tokenizer, model_lr_on_bert)
print(f"\nReview to classify: '{review_3_ai_human}'")
print(result_3)

In [ ]:
#rnn using lstm

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, SimpleRNN, Dense, SpatialDropout1D
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# --- 1. Load and Prepare the Dataset ---
try:
    df = pd.read_csv('/content/imdb_human_vs_gen_review.csv', encoding='latin-1')
except FileNotFoundError:
    print("Error: 'ai_human_text_dataset.csv' not found.")
    print("Please make sure you have created and uploaded your dataset.")
    exit()

print("Dataset Loaded Successfully:")
print(df.head())

# Ensure labels are numerical (0 and 1)
# This is crucial for the neural network's loss function
df['Label'] = df['Label'].astype(int)

# Define features (X) and target (y)
X = df['Text']
y = df['Label']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# --- 2. Tokenization and Sequencing ---
# This process converts words into integer sequences.

# Hyperparameters for tokenization
VOCAB_SIZE = 10000  # Number of unique words to consider
MAX_LENGTH = 150    # Max length of a review to consider
OOV_TOKEN = "<OOV>" # Token for words not in our vocabulary

tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token=OOV_TOKEN)
# Build the vocabulary on the training data ONLY to prevent data leakage
tokenizer.fit_on_texts(X_train)

# Convert text to sequences of integers
train_sequences = tokenizer.texts_to_sequences(X_train)
test_sequences = tokenizer.texts_to_sequences(X_test)

# --- 3. Padding ---
# All sequences must be the same length for the model. We'll pad shorter sequences and truncate longer ones.
padded_train = pad_sequences(train_sequences, maxlen=MAX_LENGTH, padding='post', truncating='post')
padded_test = pad_sequences(test_sequences, maxlen=MAX_LENGTH, padding='post', truncating='post')

print(f"\nShape of padded training data: {padded_train.shape}")
print(f"Shape of padded testing data: {padded_test.shape}")


# --- 4. Build the LSTM/RNN Model ---
EMBEDDING_DIM = 16 # Dimension of the word vectors

model = Sequential([
    # 1. Embedding Layer: Turns integer sequences into dense vector representations.
    Embedding(input_dim=VOCAB_SIZE, output_dim=EMBEDDING_DIM, input_length=MAX_LENGTH),

    # Optional: Dropout layer to prevent overfitting
    SpatialDropout1D(0.2),

    # 2. LSTM Layer: Processes the sequence of vectors.
    # LSTM is generally preferred over SimpleRNN as it handles long-term dependencies better.
    LSTM(64, dropout=0.2, recurrent_dropout=0.2),

    # To use a SimpleRNN instead, uncomment the line below and comment out the LSTM line.
    # SimpleRNN(64, dropout=0.2, recurrent_dropout=0.2),

    # 3. Dense Output Layer: Produces the final prediction.
    # Sigmoid activation is used for binary classification.
    Dense(1, activation='sigmoid')
])

# --- 5. Compile the Model ---
# We configure the model for training.
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

model.summary()

# --- 6. Train the Model ---
print("\n--- Starting Model Training ---")
NUM_EPOCHS = 5
BATCH_SIZE = 32

history = model.fit(
    padded_train,
    y_train,
    epochs=NUM_EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(padded_test, y_test),
    verbose=1
)
print("--- Training Complete ---")


# --- 7. Evaluate the Model ---
print("\n--- Evaluating Model Performance ---")
# Get model predictions (these will be probabilities)
y_pred_proba = model.predict(padded_test)
# Convert probabilities to binary class labels (0 or 1)
y_pred = (y_pred_proba > 0.5).astype(int)

print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=['Human', 'AI']))

print("\nConfusion Matrix:")
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Human', 'AI'], yticklabels=['Human', 'AI'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()


# --- 8. Function to Predict on New Text ---
def predict_rnn(text):
    # Preprocess the new text
    sequence = tokenizer.texts_to_sequences([text])
    padded_sequence = pad_sequences(sequence, maxlen=MAX_LENGTH, padding='post', truncating='post')

    # Get the prediction
    prediction_proba = model.predict(padded_sequence)[0][0]

    if prediction_proba > 0.5:
        return f"Prediction: AI (Confidence: {prediction_proba:.2f})"
    else:
        return f"Prediction: Human (Confidence: {1 - prediction_proba:.2f})"

# --- Example Predictions ---
print("\n--- Making Predictions on New Text ---")
ai_text_example = "Aashiqui 2 is a romantic musical drama that centers on the turbulent relationship between Rahul Jaykar, a famous but self-destructive singer, and Aarohi Shirke, a talented singer he discovers and mentors."
ai_humanized_text_example = "Okay, let's be real about Aashiqui 2. I still remember walking out of the theater feeling emotionally drained, and honestly, it's all because of the music. The soundtrack for this movie is legendary, hands down. You can't hear the first few notes of Tum Hi Ho without getting all the feels, am I right? Aditya Roy Kapur and Shraddha Kapoor were just perfect as RJ and Aarohi. You totally buy into their love story, which makes his struggle with alcoholism so much more painful to watch. Yeah, the plot is basically a desi A Star Is Born, and there were times I wanted to shake RJ and tell him to get it together. But the emotions and the songs are so powerful that you kind of forgive its flaws. It’s a beautiful, heartbreaking movie that will definitely stick with you."

print(f"\nInput: '{ai_text_example}'")
print(predict_rnn(ai_text_example))

print(f"\nInput: '{ai_humanized_text_example}'")
print(predict_rnn(ai_humanized_text_example))